<a href="https://colab.research.google.com/github/sreyareddy15/Mecca-Telegram-Bot/blob/main/Mecca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Mecca is a telegram bot where you can search about any function in any programming language and get information like syntax,meaning,example 

workflow is divided into 2 parts


*   Fetching data from API
*   Bot generation



#### Fetching Data from API

Here I have used syntaxdb API to fetch data

In [20]:
# importing requests HTTP library
import requests

In [21]:
# created a class to fetch json data

class Syntaxdb:
  def __init__(self,choice = 0 ,search= '',lan= ''):
    self.Choice =choice
    self.Search_query = search
    self.Language = lan
    if self.Choice == 0:
      self.url = "https://syntaxdb.com/api/v1/concepts/search?q=" +self.Search_query
    if self.Choice ==1:
      self.url = "https://syntaxdb.com/api/v1/languages/"+self.Language+"/concepts"
  def getResponse(self):
    self.results = requests.get(self.url)
    return self.results
  def getJson(self):
    self.json = self.results.json()
    return self.json

In [22]:
# created a function that converts the json data we got into a list 

def getContent(Syntaxdb):
  results = Syntaxdb.getResponse()
  json = Syntaxdb.getJson()
  summary  =[]
  # we filter the data and only take required fields of data
  key = ['concept_search','syntax','description','notes','example']
  for i in json:
    l =[]
    l = list(map(i.get,key))
    summary.append(l)
  return summary

In [40]:
# created a object and gave it to getContent() to get list data
x = Syntaxdb(0,'list in python','')
for i in getContent(x)[0]:
  print(i)

Lists in Python
list_name = [] #empty list to start
list_name2 = [item1, item2] #non empty list to start

list_name.append(item) #add to the list

list_name[index] #access the item at the list index
Lists are used to store multiple items sequentially in one variable. Equivalent of an array in other languages. Lists are mutable.
Items inside of the list can be of different types. Data structures and other lists can also be stored within a list.

The array index range is from 0 to size minus 1. Negative indices start at the end of the array and work backwards.

Lists can be added (using addition operator) or concatenated (using the extend function). There are many other list functions, which can be found in the PyDocs.
cities = ["Toronto", "San Francisco", "Seattle"]

print (cities[0]) #prints Toronto


# Telegram bot 
( generating + integration + deployment )

In [24]:
# use this command if you don't have it preinstalled
# %pip install python-telegram-bot

In [25]:
# all required functions and libraries
from telegram.ext import Updater
from telegram.ext.commandhandler import CommandHandler
from telegram.ext.callbackcontext import CallbackContext
from telegram import Bot
from time import sleep
from telegram.update import Update
from telegram.ext.dispatcher import Dispatcher

In [26]:
# your API token that you get from botfather
key = 'API token'

In [27]:
# make a bot using your key
# get me function will help you get basic information about your bot
bot = Bot(key)
print(bot.get_me())

{'id': 1262656577, 'first_name': 'Mecca', 'is_bot': True, 'username': 'in_all_bot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': False}


In [28]:
# provides a frontend to telegram.Bot to the programmer, so they can focus on coding the bot. 
#Its purpose is to receive the updates from Telegram and to deliver them to said dispatcher
updater = Updater(key,use_context=True)
dispatcher :Dispatcher  = updater.dispatcher

In [29]:
# start function 
def start(update : Update, context : CallbackContext):
    #bot.send_chat_action(chat_id= update.effective_chat.id,action = ChatAction.TYPING)
    bot.send_message(chat_id=update.effective_chat.id, text="Welcome to Mecca")

In [30]:
#help function
def help(update:Update,context:CallbackContext):
  #bot.send_chat_action(chat_id= update.effective_chat.id,action = ChatAction.TYPING)
  text = "To use this bot,look at the format below!\nsearch<concept_name><language>\nExample: Suppose you want to know how to use the concept for in java \n /search for in java."
  bot.send_message(chat_id=update.effective_chat.id,text=text)

In [31]:
# search function
def search(update:Update,context:CallbackContext):
  arg = " ".join(context.args)
  # if search query is empty
  if arg == " ":
    text = "please make sure to enter valid search for me to help out"
    bot.send_message(chat_id=update.effective_chat.id,text=text)
  # creating an object of Syntaxdb with giving query
  x = Syntaxdb(choice = 0 ,search=arg,lan='')
  # fetching list data
  # even though we get list of many functions , we fetch the first most appropriate one ( we can change number of lists we want )
  result = getContent(x)[0]
  # sending list data to bot
  for i in result:
    bot.send_message(chat_id=update.effective_chat.id,text=i)
  #if we don't find any results
  if result == []:
    bot.send_message(chat_id=update.effective_chat.id,text='there are no related searches to topic you entered!Make sure your search query was right and try again! check for spelling mistakes and follow the format in help')

In [32]:
# it gets the command we got from bot and call back appropriate function 
dispatcher.add_handler(CommandHandler('start', start))
dispatcher.add_handler(CommandHandler('help', help))
dispatcher.add_handler(CommandHandler('search',search,pass_args=True))

In [33]:
## Starts polling updates from Telegram.
updater.start_polling()